<a href="https://colab.research.google.com/github/oimartin/SP_500_index_RNN/blob/main/sp500_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ingest

## Load Libraries

In [17]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
%matplotlib inline

## Load Data

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/oimartin/SP_500_index_RNN/main/data/sp_500_5yr_02282017_01282022.csv')

In [3]:
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,28-Feb-22,"4,354.17","4,388.84","4,315.12","4,373.94","4,373.94","4,594,010,000"
1,25-Feb-22,"4,298.38","4,385.34","4,286.83","4,384.65","4,384.65","3,941,780,000"
2,24-Feb-22,"4,155.77","4,294.73","4,114.65","4,288.70","4,288.70","5,070,560,000"
3,23-Feb-22,"4,324.93","4,341.51","4,221.51","4,225.50","4,225.50","3,814,340,000"
4,22-Feb-22,"4,332.74","4,362.12","4,267.11","4,304.76","4,304.76","4,007,780,000"


# EDA

## Pre-processing

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1259 entries, 0 to 1258
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Date       1259 non-null   object
 1   Open       1259 non-null   object
 2   High       1259 non-null   object
 3   Low        1259 non-null   object
 4   Close      1259 non-null   object
 5   Adj Close  1259 non-null   object
 6   Volume     1259 non-null   object
dtypes: object(7)
memory usage: 69.0+ KB


In [5]:
data = data.replace(',','', regex=True)
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,28-Feb-22,4354.17,4388.84,4315.12,4373.94,4373.94,4594010000
1,25-Feb-22,4298.38,4385.34,4286.83,4384.65,4384.65,3941780000
2,24-Feb-22,4155.77,4294.73,4114.65,4288.70,4288.70,5070560000
3,23-Feb-22,4324.93,4341.51,4221.51,4225.50,4225.50,3814340000
4,22-Feb-22,4332.74,4362.12,4267.11,4304.76,4304.76,4007780000


In [6]:
data.iloc[:, 1:] = data.iloc[:, 1:].astype('float64')
data['Date'] = pd.to_datetime(data['Date'])
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-02-28,4354.17,4388.84,4315.12,4373.94,4373.94,4594010000.0
1,2022-02-25,4298.38,4385.34,4286.83,4384.65,4384.65,3941780000.0
2,2022-02-24,4155.77,4294.73,4114.65,4288.7,4288.7,5070560000.0
3,2022-02-23,4324.93,4341.51,4221.51,4225.5,4225.5,3814340000.0
4,2022-02-22,4332.74,4362.12,4267.11,4304.76,4304.76,4007780000.0


In [7]:
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-02-28,4354.17,4388.84,4315.12,4373.94,4373.94,4594010000.0
1,2022-02-25,4298.38,4385.34,4286.83,4384.65,4384.65,3941780000.0
2,2022-02-24,4155.77,4294.73,4114.65,4288.7,4288.7,5070560000.0
3,2022-02-23,4324.93,4341.51,4221.51,4225.5,4225.5,3814340000.0
4,2022-02-22,4332.74,4362.12,4267.11,4304.76,4304.76,4007780000.0


## First View

In [8]:
fig = go.Figure(data=[go.Candlestick(x=data['Date'], close=data['Close'], open=data['Open'], 
                             low=data['Low'], high=data['High'])])

fig.show()

In [16]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=data['Volume']))

# Overlay both histograms
fig.update_layout(bargap=0.1)
# Reduce opacity to see both histograms
fig.show()

In [13]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=data['Open']))
fig.add_trace(go.Histogram(x=data['Close']))

# Overlay both histograms
fig.update_layout(barmode='overlay', bargap=0.1)
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.6)
fig.show()

In [35]:
data['Year'] = data['Date'].dt.strftime('%Y')
data['Month'] = data['Date'].dt.strftime('%m')
data['Day'] = data['Date'].dt.strftime('%d')

In [37]:
data

,Date,Open,High,Low,Close,Adj Close,Volume,Year,Month,Day
0,2022-02-28,4354.17,4388.84,4315.12,4373.94,4373.94,4594010000.0,2022,02,28
1,2022-02-25,4298.38,4385.34,4286.83,4384.65,4384.65,3941780000.0,2022,02,25
2,2022-02-24,4155.77,4294.73,4114.65,4288.7,4288.7,5070560000.0,2022,02,24
3,2022-02-23,4324.93,4341.51,4221.51,4225.5,4225.5,3814340000.0,2022,02,23
4,2022-02-22,4332.74,4362.12,4267.11,4304.76,4304.76,4007780000.0,2022,02,22
...,...,...,...,...,...,...,...,...,...,...
1254,2017-03-07,2370.74,2375.12,2365.51,2368.39,2368.39,3518390000.0,2017,03,07
1255,2017-03-06,2375.23,2378.8,2367.98,2375.31,2375.31,3232700000.0,2017,03,06
1256,2017-03-03,2380.92,2383.89,2375.39,2383.12,2383.12,3555260000.0,2017,03,03
1257,2017-03-02,2394.75,2394.75,2380.17,2381.92,2381.92,3821320000.0,2017,03,02


In [67]:
m_y_data = data.groupby(['Month', 'Year']).median().reset_index()

def by_year (df):
  m_y_data.loc[m_y_data['Year']== '2022'].sort_values(by='Month', ascending=True).Open
  return


yr2022 = m_y_data.loc[m_y_data['Year']== '2022'].sort_values(by='Month', ascending=True).Open
yr2021 = m_y_data.loc[m_y_data['Year']== '2021'].Open
yr2020 = m_y_data.loc[m_y_data['Year']== '2020'].Open
yr2019 = m_y_data.loc[m_y_data['Year']== '2019'].Open
yr2018 = m_y_data.loc[m_y_data['Year']== '2018'].Open
yr2017 = m_y_data.loc[m_y_data['Year']== '2017'].Open

In [73]:
m_y_data.loc[m_y_data['Year']== '2018'].sort_values(by='Month', ascending=True)

,Month,Year,Date,Open,High,Low,Close,Adj Close,Volume,Day
0,01,2018,2018-01-17 00:00:00,2798.96,2807.040,2778.380,2798.03,2798.03,3.576350e+09,17.0
5,02,2018,2018-02-14 00:00:00,2715.80,2737.600,2697.770,2703.96,2703.96,3.938450e+09,14.0
11,03,2018,2018-03-15 00:00:00,2715.05,2730.890,2701.740,2716.94,2716.94,3.500330e+09,15.0
16,04,2018,2018-04-16 00:00:00,2657.36,2676.480,2647.160,2656.87,2656.87,3.349370e+09,16.0
21,05,2018,2018-05-15 12:00:00,2713.30,2724.305,2703.485,2716.55,2716.55,3.349680e+09,15.5
26,06,2018,2018-06-15 00:00:00,2760.79,2769.280,2748.460,2762.59,2762.59,3.555090e+09,15.0
31,07,2018,2018-07-17 00:00:00,2797.36,2808.610,2793.390,2801.83,2801.83,3.063850e+09,17.0
36,08,2018,2018-08-16 00:00:00,2855.92,2862.440,2851.980,2856.98,2856.98,2.976970e+09,16.0
41,09,2018,2018-09-17 00:00:00,2903.83,2908.300,2895.770,2904.31,2904.31,3.241250e+09,17.0
46,10,2018,2018-10-16 00:00:00,2775.66,2797.770,2755.180,2767.78,2767.78,3.598710e+09,16.0


In [68]:
years = ['2021', '2020', '2019', '2018']
colors = ['slategray', 'magenta', 'red', 'green']
fig = ff.create_distplot([yr2021, yr2020, yr2019, yr2018], years, curve_type='normal', colors=colors)
fig.show()

In [38]:
data.groupby([data['Date'].dt.month_name()]).median()


,Date,Open,High,Low,Close,Adj Close,Volume,Year,Month,Day
Date,,,,,,,,,,
April,2019-04-16 00:00:00,2799.340,2818.570,2775.950,2799.310,2799.310,3.635030e+09,2019.0,4.0,16.0
August,2019-08-14 00:00:00,2901.450,2916.500,2893.500,2901.130,2901.130,3.158450e+09,2019.0,8.0,16.0
December,2019-12-18 12:00:00,3195.305,3201.980,3191.730,3198.945,3198.945,3.574245e+09,2019.0,12.0,15.5
February,2020-02-14 00:00:00,3335.540,3347.960,3322.120,3337.750,3337.750,3.871340e+09,2020.0,2.0,14.0
January,2020-01-14 12:00:00,3268.580,3284.385,3257.040,3274.050,3274.050,3.727025e+09,2020.0,1.0,16.0
July,2019-07-17 12:00:00,3001.490,3005.960,2986.590,2997.865,2997.865,3.237055e+09,2019.0,7.0,17.0
June,2019-06-17 00:00:00,2903.270,2910.610,2887.300,2891.640,2891.640,3.651640e+09,2019.0,6.0,15.0
March,2019-03-16 12:00:00,2751.740,2765.140,2734.215,2746.945,2746.945,3.905850e+09,2019.0,3.0,16.0
May,2019-05-13 12:00:00,2831.220,2852.670,2810.285,2832.560,2832.560,3.649720e+09,2019.0,5.0,15.0


In [10]:
px.density_contour(data, x=['Open', 'Close', 'High', 'Low'])